In [1]:
import os
import glob
import pandas as pd

In [6]:
# Specify the name of the padlock pool

padlock_pool = 'taar_genes'

# Read in all csvs that begin with "6.Probes" and end with ".csv"

# set the directory where the subdirectories are located
parent_dir = '/nemo/lab/znamenskiyp/scratch/'

# create an empty list to hold the csv files
csv_files = []

# loop through all subdirectories that start with 'olf_chunk_'
for subdir in [d for d in os.listdir(parent_dir) if d.startswith(f'{padlock_pool}_chunk_')]:
    subpath = os.path.join(parent_dir, subdir)
    
    # check if the subdirectory contains a csv file that starts with '6.ProbesRandom'
    if os.path.isdir(subpath) and glob.glob(os.path.join(subpath, '6.ProbesRandom*.csv')):
        
        # if a matching csv file is found, add its path to the list
        csv_files.append(glob.glob(os.path.join(subpath, '6.ProbesRandom*.csv'))[0])
        
# read in the first csv file to get the header
df = pd.read_csv(csv_files[0])

# loop through the remaining csv files and concatenate them into one dataframe
for csv_file in csv_files[1:]:
    temp_df = pd.read_csv(csv_file)
    df = pd.concat([df, temp_df], ignore_index=True)

df = df[~df['acronym'].str.startswith('>')]
df = df.dropna()

df['gene_name'] = df['acronym'].str.extract(r'\((.*?)\)')

# define a function to remove characters from a string
def remove_chars(s, indices):
    for i in sorted(indices, reverse=True):
        s = s[:i] + s[i+1:]
    return s

# define the indices of the characters to remove
indices_to_remove = [20, 21, 61, 62]

# apply the function to the 'padlock' column of the dataframe
df['padlock'] = df['padlock'].apply(lambda x: remove_chars(x, indices_to_remove))


# save the merged dataframe to a csv file
df.to_csv(f'{padlock_pool}_probes.csv', index=False)